# 27 Pandas Functions You Didn't Know Existed | P(Guarantee) = 0.9
## Making Pandas Even More Awesome
![](https://images.pexels.com/photos/5199661/pexels-photo-5199661.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@introspectivedsgn?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Erik Mclean</a>
        on 
        <a href='https://www.pexels.com/photo/unrecognizable-man-in-panda-head-sitting-near-car-5199661/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels.</a> All images are by the author unless specified otherwise.
    </strong>
</figcaption>

## Setup

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

## Introduction

## 1. ExcelWriter

`ExcelWriter` is a generic class for creating excel files (with sheets!) and writing DataFrames to them. Let's say we have these 2:

In [2]:
diamonds = sns.load_dataset("diamonds")
tips = sns.load_dataset("tips")

with pd.ExcelWriter("data/data.xlsx") as writer:
    diamonds.to_excel(writer, sheet_name="diamonds")
    tips.to_excel(writer, sheet_name="tips")

It has additional attributes to specify the datetime format to be used, whether you want to create a new excel file or modify an existing one, what happens when a sheet exists, etc. Check out the details from the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.ExcelWriter.html).

## 2. factorize

This function is a pandas alternative to Sklearn's `LabelEncoder`:

In [6]:
codes, unique = pd.factorize(diamonds["cut"], sort=True)

codes[:10]

array([0, 1, 3, 1, 3, 2, 2, 2, 4, 2], dtype=int64)

In [7]:
unique

CategoricalIndex(['Ideal', 'Premium', 'Very Good', 'Good', 'Fair'], categories=['Ideal', 'Premium', 'Very Good', 'Good', 'Fair'], ordered=False, dtype='category')

Pay attention to how the function returns the categories as well. 

In [10]:
diamonds["cut_enc"] = pd.factorize(diamonds["cut"])[0]
diamonds["cut_enc"].sample(5)

46228    0
10292    3
50925    1
38309    1
1775     0
Name: cut_enc, dtype: int64

## 3. bdate_range

A short-hand function to create TimeSeries indices with business-day frequency:

In [14]:
series = pd.bdate_range("2021-01-01", "2021-01-31")  # A period of one month
len(series)

21

Business-day frequencies are common in the financial world. So, this function may come in handy when reindexing existing time-series with `reindex` function.

## 4. hasnans

Pandas offers a quick method to check if a given series contains any nulls with `hasnans` attribute:

In [15]:
series = pd.Series([2, 4, 6, "sadf", np.nan])
series.hasnans

True

According to its [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.hasnans.html), it enables various performance increases. Note that the attribute is pd.Series-only.

## 5. convert_dtypes

We all know that pandas has an annoying tendency to mark some columns as `object` data type. Instead of manually specifying their types, you can use `convert_dtypes` method which infers the best data type:

In [25]:
sample = pd.read_csv(
    "data/station_day.csv",
    usecols=["StationId", "CO", "O3", "AQI_Bucket"],
)
sample.dtypes

StationId      object
CO            float64
O3            float64
AQI_Bucket     object
dtype: object

In [28]:
sample.convert_dtypes().dtypes

StationId      string
CO            float64
O3            float64
AQI_Bucket     string
dtype: object

Unfortunately, it can't pares dates due to the caveats of different date time formats.

## 6. `at` and `iat`

These two accessors are much faster alternatives to `loc` and `iloc` with a disadvantage. They only allow selecting or replacing a single value at a time:

In [29]:
diamonds.at[234, "cut"]

'Ideal'

In [30]:
diamonds.iat[1564, 4]

61.2

In [31]:
# Replace 16541th row of the price column
diamonds.at[16541, "price"] = 10000

## 7. `min` and `max` along the columns axis

Even though `min` and `max` functions are well-known, they have another useful property for some edge-cases. Consider this dataset:

In [34]:
index = ["Diamonds", "Titanic", "Iris", "Heart Disease", "Loan Default"]
libraries = ["XGBoost", "CatBoost", "LightGBM", "Sklearn GB"]
df = pd.DataFrame(
    {lib: np.random.uniform(90, 100, 5) for lib in libraries}, index=index
)

df

,XGBoost,CatBoost,LightGBM,Sklearn GB
Diamonds,99.733903,96.540712,91.725201,93.640438
Titanic,93.130239,99.838026,93.586711,95.963391
Iris,97.153966,98.354851,90.585250,93.693119
Heart Disease,92.590182,94.402339,93.373349,93.399484
Loan Default,98.359192,91.073934,90.595855,92.900320


The above fake DataFrame is a point-performance of 4 different gradient boosting libraries on 5 datasets. We want to find the library that performed best at each dataset non-manually. Here is how you do it with `max`:

In [37]:
df.max(axis=1)

Diamonds         99.733903
Titanic          99.838026
Iris             98.354851
Heart Disease    94.402339
Loan Default     98.359192
dtype: float64

Just change the axis to 1 and you get a row-wise max/min. 

## 8. pipe

This is one of the best functions for doing data cleaning in a concise, compact manner. `pipe` function allows you to chain multiple custom functions into a single operation.

For example, let's say you have custom functions to `drop_duplicates`, `remove_outliers`, `encode_categoricals` that accept their own custom arguments. Here is how you apply all three in a single operation:

```python
df_preprocessed = (diamonds.pipe(drop_duplicates).
                            pipe(remove_outliers, ['price', 'carat', 'depth']).
                            pipe(encode_categoricals, ['cut', 'color', 'clarity'])
                  )
```

I like how this function resembles [Sklearn pipelines](https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d). There is more you can do with the function, so check out the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html) or this [helpful article](https://towardsdatascience.com/a-better-way-for-data-preprocessing-pandas-pipe-a08336a012bc).